# Generalizing to Higher Dimensions

## Intro

Linear algebra generalizes everything we know about 2D and 3D geometry to study data in any number of dimensions. This means that the machinery we've see on vectors, linear transformations, and matrices can still be useful.

In this chapter we'll define **vector spaces**. Vector spaces are collections of objects we can treat like vectors. These can be arrows in the plane, tuples of numbers, or objects completely different from the ones we've used so far. For example, you can treat images as vectors and take a linear combination of them.

The key operations in a vector space are vector addition and scalar multiplication. With these you can make linear combinations (negation, subtraction, weighted averages, ...) and you can reason about which transformations are linear.

We'll also deep dive on the meaning of the word dimension, and we'll see that images are 270,000 dimensional objects.

## Generalizing our definition of vectors

Python is an object-oriented language, and we can use that for generalization. Through inheritance, you can create new classes of object that inherit properties and behaviors from an existing parent class.

| NOTE: |
| :---- |
| We already used inheritance in our `vec2d.graph` and `vec3d.graph` libraries to reduce the amount of code for drawing the different elements, and also to make the code more succinct. |

### Creating a class for 2D coordinate vectors

Here we include the implementation notes for the code in [12: Vec class library](../../02_mini-projects/12-vec-class-lib/README.md).


We create a class `Vec2` in which we define the vector operations as class methods.

We foster an immutable approach. For example, `v.add(w)` returns a new vector rather than modifying the one in which add is invoked.

By default, Python compares instances by their references rather than their values. In order to make comparisons more sensible, we override the `==` operator so that we can compare vectors by value. Ultimately, this means overriding the `__eq__` method.

As Python supports operator overloading, it is only natural to make use of it for vectors. Thus, we will be able to do `v + w`, `w * s` and `s * w` instead of the equivalent `v.add(w)` and `v.scale(s)`. In order to do so, you just need to override the methods: `__add__`, `__mul__` (when the class in on the left of the expression `w * s`), and `__rmul__` (when the class in on the right of the expression `s * w`).

Also, we implement a user-friendly description for class instances:
+ `__repr__` is intended for debugging and development.
+ `__str__` is intended for showing a descriptive information intended for users of the code.

### Repeating the process with 3D vectors

We can right away repeat the approach for 3D vectors.

With 2D and 3D implemented we can start thinking about generalizations. While there are many ways to do so, we want to generalize *how we use the vectors* rather than focusing on code reuse, or simplifying certain arithmetic operations.

For example, after the generalization we want to be able to write:

```python
def average(v1, v2):
  return 0.5 * v1 + 0.5 * v2
```

And we don't want the user of our library to specify whether `v1` and `v2` are 2D or 3D vectors. In fact, we want to use it with images!

### Building a Vector base class

To create the vector base class use:

```python
from abc import ABC
class Vector(ABC):
```

Decorate abstract methods in the base class with:

```python
from abc import ABC, abstractmethod

@abstractmethod
def scale(self, scalar):
  pass
```

An abstract class with abstract methods cannot be instantiated &mdash; you'll get a `TypeError` if you try to.

With the abstract methods defined, you can define a few other methods that depend on the abstract ones, like `__add__`, `__mul__`, and `__rmul__`.

Then you can simplify the implementation of `Vec2` and `Vec3` class to benefit from the base class, and remove the implementation of `__add__`, `__mul__`, and `__rmul__` which will be inherited from the base class.

Now we can start adding methods that we think will be useful, such as `subtract` and `__sub__`.


### Defining Vector Spaces

In Math, a vector is defined by what it does, rather than what it is.

For example, a vector must be object equipped with a *suitable* way to add it to other vectors and multiply it by scalars.

By *suitable* we mean a few rules these operations must comply with.

Specifically:

1. Adding vectors in any order shouldn't matter: $ \vec{v} + \vec{w} = \vec{w} + \vec{v} $ for any vectors $ \vec{v} $ and $ \vec{w} $.

2. Adding vectors in any grouping shouldn't matter: $ \vec{u} + (\vec{v} + \vec{w}) = (\vec{u} + \vec{v}) + \vec{w} $. As a corolary, $ 3 \vec{v} = \vec{v} + \vec{v} + \vec{v} $.

3. If $ a $ and $ b $ are scalars and $ \vec{v} $ is a vector, then $ a \cdot (b \cdot \vec{v}) $ should be the same as $ (a \cdot b) \cdot \vec{v} $.

4. Multiplying a vector by 1 should leave it unchanged: $ 1 \cdot \vec{v} = \vec{v} $.

5. Addition of scalars should be compatible with scalar multiplication: $ a \cdot \vec{v} + b \cdot \vec{v} = (a + b) \cdot \vec{v} $.

6. Addition of vectors should be compatible with scalar multiplication: $ a \cdot (\vec{v} + \vec{w}) $ should be the same as $ a \cdot \vec{v} + a \cdot \vec{w} $.

A vector space is a collection of objects called vectors, equipped with suitable vector addition and scalar multiplication operations obeying the six rules stated above, such that every linear combination of vectors in the collection produces a vector that is also in the collection.

For example the collection: `[Vec2(1, 0), Vec2(5, -3), Vec2(1.1, 0.8)]` is a group of vectors that can be added and multiplied, but they do not form a vector space because:

```python
1 * Vec2(1, 0) + 1 * Vec2(5, -3) = Vec2(6, -3)
```

And `Vec(6, -3)` is not part f the collection.

By contrast, the collection of all possible 2D vectors is a vector space.

There are two implications of the fact that vector spaces need to contain all their scalar multiples, and these implications are important enough to mention.

1. No matter what vector $ \vec{v} $ you pick in a vector space, $ 0 \cdot \vec{v} $ yields the *zero vector* denoted as $ \vec{0} $. Adding the zero vector to any vector leaves the vector unchanged: $ \vec{0} + \vec{v} = \vec{v} + \vec{0} = \vec{v} $.

2. Every vector has an opposite vector $ -1 \cdot \vec{v} $. This is because of rule #5, $ \vec{v} + \vec{-v} = (1 - 1) \cdot \vec{v} = 0 \cdot \vec{v} = \vec{0} $.

### Unit testing vector space classes

In Math, the usual way to guarantee *suitability* is by writing a proof. In Python, we do so by writing unit tests.

In Python, we also typically rely on brute force provided by CPU rathern than finding an *algebraic proof* as we do in Math. For example, we can rely on getting random scalars and vectors to check certain rules:

In [2]:
from random import uniform
from vec2 import Vec2

def random_scalar():
    return uniform(-10, 10)

def random_vec2():
    return Vec2(random_scalar(), random_scalar())

a = random_scalar()
u, v = random_vec2(), random_vec2()

assert a * (u + v) == a * u + a * v

AssertionError: 

While intuition tells us that the previous snippet should work ok it will most surely fails.

This happens because vector coordinates will be represented as floats, and they will differ by a small amount.

We can get rid of the error using `math.isclose`

In [3]:
from random import uniform
from math import isclose
from vec2 import Vec2

def random_scalar():
    return uniform(-10, 10)

def random_vec2():
    return Vec2(random_scalar(), random_scalar())

def approx_equal_vec2(u, v):
    return isclose(u.x, v.x) and isclose(u.y, v.y)

a = random_scalar()
u, v = random_vec2(), random_vec2()

assert approx_equal_vec2(a * (u + v), a * u + a * v)

Again, testing this by a single sample doesn't look like a strong enough proof. But we can use the power of the CPU to test by a large number of samples and for all the six rules:

In [19]:
from random import uniform
from math import isclose
from vec2 import Vec2
import time

def random_scalar():
    return uniform(-10, 10)

def random_vec2():
    return Vec2(random_scalar(), random_scalar())

def approx_equal_vec2(u, v):
    return isclose(u.x, v.x) and isclose(u.y, v.y)

def test_space_vector_rules(eq, a, b, u, v, w):
    assert eq(u + v, v + u)
    assert eq(u + (v + w), (u + v) + w)
    assert eq(a * (b * v), (a * b) * v)
    assert eq(1 * v, v)
    assert eq((a + b) * v, a * v + b * v)
    assert eq(a * v + a * w, a * (v + w))

start_t = time.time()

for _ in range(0, 100):
    a, b = random_scalar(), random_scalar()
    u, v, w = random_vec2(), random_vec2(), random_vec2()
    test_space_vector_rules(approx_equal_vec2, a, b, u, v, w)
print(f"Test took {(time.time() - start_t):.3f} sec")

Test took 0.001 sec


It took less that 1 thousand of a second to test 100 samples. We could typically allow then to run more than 1000 samples and it will still be a very quick test. 

Testing with these sort of assertions are OK for Jupyter notebooks, but in programs you almost always rely on a unit test framework like [unittest](https://docs.python.org/3/library/unittest.html) (the Unit Test framework that comes with Python stdlib) or PyTest.

You can review the test classes from [12: Vec class library](../../02_mini-projects/12-vec-class-lib/README.md) for examples using [unittest](https://docs.python.org/3/library/unittest.html).

### Exercise

Implement a `CoordinateVector` class inheriting from `Vector` with an abstract property representing the dimension. This should save repetitious work when implementing specific coordinate vector classes. Inheriting from `CoordinateVector` and setting the dimension to `6` should be all you need to implement a `Vec6` class.

The implementation can be found in [CoordinateVector](../../02_mini-projects/12-vec-class-lib/coordvec/coordveclib.py).

The relevant details of the implementation follows.

The implementation relies on the tuple representation of vectors that we used in `vec3d.math`.

One possible way to represent the dimension is:

```python
from abc import abstractmethod

class CoordinateVector(Vector):
    @classmethod
    @abstractmethod
    def dimension(cls):
        pass
```

While the book suggests this could be implemented as an instance property, but as all the vectors of a given concrete class will have the same dimension, it feels more reasonable to use a class method.

When implementing the `scale` and `add` methods, you need to solve the following challenge:
> You need return an instance of the corresponding class, but you cannot return `CoordinateVector` because it is an abstract class.

In Python, you can do that using:

```python
return self.__class__(<coords>)
```

That will effectively instantiate the concrete subclass.

Another challenge is the implementation of the `__repr__` method, because ideally we should be following the `Vec2(x, y)` approach.

That is possible using `self.__class__.__qualname__`:

```python
def __repr__(self):
    return f"{self.__class__.__qualname__}{self.coordinates}"
```

With the implementation in place you'll be able to do:

In [2]:
from coordvec import CoordinateVector

class Vec6(CoordinateVector):
    @classmethod
    def dimension(cls):
        return 6

u = Vec6(1, 2, 3, 4, 5, 6)
v = Vec6(11, 12, 13, 14, 15, 16)

print(u + v)
print(10 * u)

(12, 14, 16, 18, 20, 22)
(10, 20, 30, 40, 50, 60)


### Exercise

Add a `zero` abstract method to the `Vector` class to return the zero vector in a given vector space, as well as an implementation for the negation operator.

These are useful because we're required to have a zero vector and negations of any vector in a vector space.

The solution is implemented in [12: Vector class library](../../02_mini-projects/12-vec-class-lib/).

These are the details.

In the `Vector` class we define the `zero` abstract class method and we implement the `__neg__` operator banking on the capabilities already defined:

```python
class Vector(ABC):
    """Abstract class for vectors"""

    @classmethod
    @abstractmethod
    def zero(cls):
        ...

...
    def __neg__(self):
        return self.scale(-1)
```

While the exercise suggests we should be defining `zero` as a property, the latest documentation states that it is no longer possible to do so:
> Changed in version 3.11: Class methods can no longer wrap other descriptors such as property()

See https://docs.python.org/3.11/library/functions.html#classmethod

Note that the only minor change is that `zero` will have to be invoked with parentheses, as it is method and not a property.

Then, we can provide the implementation of `zero` in our examples `vec2`, `vec3`, and `CoordinateVector`.
In particular, the implementation for `CoordinateVector` can be concrete:

```python
class CoordinateVector(Vector):

    @classmethod
    def zero(cls):
        return cls(*tuple(0 for _ in range(cls.dimension())))
```

### Exercise

Write unit tests to show that the addition and scalar multiplication operations defined for `Vec3` satisfy the vector space properties.

You can find the unit tests, with a little bit of refactoring, on [12: Vector class library](../../02_mini-projects/12-vec-class-lib/).

In the example, you can see how a helper class `TestUtils` was created to collect some common codes.

### Exercise

Add unit tests to check that $ \vec{0} + \vec{v} = \vec{v} $, $ 0 \cdot \vec{v} = \vec{0} $, and $ -\vec{v} + \vec{v} = \vec{0} $.

With the refactoring introduced in the previous exercise, we just need update the `check_vector_space_rules` method to add the new rules:

```python
class TestUtils(unittest.TestCase):
    """Utility methods"""
...

    def check_vector_space_rules(self, eq_fn, a, b, u, v, w):
        # Required for vector spaces
        self.assertTrue(eq_fn(u + v, v + u))
        self.assertTrue(eq_fn(u + (v + w), (u + v) + w))
...
        # Corollaries
        zero_v = u.__class__.zero()
        self.assertTrue(eq_fn(zero_v + u, u))
        self.assertTrue(eq_fn(0 * u, zero_v))
        self.assertTrue(eq_fn(-v + v, zero_v))
```

The only interesting point is that because `zero()` is a class method you cannot use polymorphism right away.

However, you can use the trick:

```python
zero_v = u.__class__.zero()
```

to obtain a representation of the zero vector for the corresponding class of `u`.

### Exercise

The implementation of the equality function is too forgiving, making `Vec2(1, 2) == Vec3(1, 2, 3)` returns `True`. Fix this by adding a check so that classes match before testing for vector equality. Fix also the approx. equal check used in the tests.

Additional guards need to be added to the different classes to check that vector classes added or compared are compatible.

For `Vec2` and `Vec3` is easy:

```python
    def add(self, other):
        if not isinstance(other, Vec2):
            raise TypeError("Incompatible vectors")
        return Vec2(self.x + other.x, self.y + other.y)

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__
            and self.x == other.x
            and self.y == other.y
        )
```

For `CoordinateVector` is a bit more complex, as the comparison `self.class == other.__class__` was failing in the tests for compatible classes.

Therefore, the following guards were added:

```python
def add(self, other):
    if not isinstance(other, CoordinateVector) or len(
        self.coordinates
    ) != len(other.coordinates):
        raise TypeError("Incompatible vectors")
    return self.__class__(*add(self.coordinates, other.coordinates))

def __eq__(self, other):
    if not isinstance(other, CoordinateVector) or len(
        self.coordinates
    ) != len(other.coordinates):
        return False
    return all(
        [
            coord_u == coord_v
            for coord_u, coord_v in zip(self.coordinates, other.coordinates)
        ]
    )
```

### Exercise

Implement a `__truediv__` function on `Vector` that allows you to divide vectors by scalars. You can divide vectors by a non-zero scalar by multiplying them by the reciprocal of the scalar `(1.0 / scalar)`.

## Exploring different vector spaces